https://gitcode.csdn.net/65ea8a491a836825ed7946ff.html?spm=1001.2101.3001.6650.15&utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Eactivity-15-134085503-blog-148143474.235%5Ev43%5Epc_blog_bottom_relevance_base8&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Eactivity-15-134085503-blog-148143474.235%5Ev43%5Epc_blog_bottom_relevance_base8

# 位置编码
transformer 架构的特点：注意力机制是”位置无关的",不加入位置信息，计算时是无法考虑到位置信息的。
注意力分数仅由两个向量的内积（向量相似度）决定，与这两个向量在序列中的位置无关。例如以下两句话："他欠我100万","我欠他100万". 若不添加位置编码，两句话中  “我” 和“他”  ，“他” 和“我” 的内积是一样的，导致计算得到的注意力得分是相同的，导致加权之后“我”和“他”的语义是完全一样的，从而导致模型无法区分语义相反的两个句子。

位置编码就是为了打破这种对称性，使得相同的token 由于位置不同，语义也有所区别

## 1.三角函数位置编码
transformer论文中提出的就是这种位置编码方式
\begin{array}{l}
P E_{(p o s, 2 i+1)}=\cos \left(\frac{p o s}{10000 \frac{2 i}{d_{\text {model }}}}\right) \\
P E_{(p o s, 2 i)}=\sin \left(\frac{p o s}{10000 \frac{2 i}{d_{\text {model }}}}\right)
\end{array}

其中，pos指的是token在序列中的位置。例如：“我爱你” 我的pos 是0，爱的pos是1
2i 和2i+1 指的是词向量的每一个维度。如果词向量维度d_model=768,2i指的是（0，2，...,766）,2i+1指的是（1，3，...,767), i是（0，1，2，..., d_model//2)

在编程实现中：公式中的幂计算通常变成先求对数再求指数

In [ ]:
import torch.nn  as nn
import torch
class Position_encoding(nn.Module):
    def __init__(self,d_model:int ,max_len:int,dropout:float):
        '''
        d_model:嵌入维度
        max_len:序列长度
        '''
        super().__init__()
        self.dropout = nn.Dropout(dropout)

        pe = torch.zeros(max_len,d_model) #存放位置编码
        position = torch.arange(0,max_len-1).unsqueeze(1)
        # 定义div_term, 用于缩放不同位置的正余弦
        div_term = torch.exp(torch.arange(0,d_model,2) * -(torch.log(10000) /d_model))

        pe[:,0::2] = torch.sin(position * div_term)
        pe[:,1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)

        self.register_buffer("pe",pe)

    def forward(self, x):
        pass
